# Generating Plots for Romance Movies

In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
!pip install nltk

import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd
import time
import nltk
from nltk import tokenize
nltk.download('punkt')

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Utility Functions

In [0]:
#model run name
romance_model = 'run_medium_romancefinal_2k'

In [0]:
#single generation
def generate_plot(prefix, model_run_name, size = 100):
  output = gpt2.generate(sess,
                run_name=model_run_name,
                prefix = prefix,
                length=size,
                temperature=0.7,
                top_p=0.9,
                return_as_list=True
                )
  out = output[0].rpartition('.')[0] + "."
  return out

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name=romance_model)

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=romance_model)

Loading checkpoint checkpoint/run_medium_romancefinal_2k/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/run_medium_romancefinal_2k/model-2000


### Sample Generation

In [0]:
generate_plot("Bella is killed by her husband, one dark and stormy night.", romance_model, 50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


'Bella is killed by her husband, one dark and stormy night. She is left with her two sons, who are now in love with each other, and a daughter, who is in love with the boy she left for.'

In [0]:
generate_plot("Bella marries the man of her dreams last summer on a boat.", romance_model, 50)

'Bella marries the man of her dreams last summer on a boat. But the real boyfriend, a rich businessman, is in love with Bella. Bella is heartbroken and leaves town.'

In [0]:
generate_plot("Bella is killed by her husband, one dark and stormy night.", romance_model)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


"Bella is killed by her husband, one dark and stormy night. Angered by the death, Bella goes to her mother's house and is found dead. She is taken away by the police, but she is found to be pregnant. She is given a gun and she shoots herself in the head. Bella is brought to the hospital, and her death is recorded. Bella is then released and her ex-husband, a detective, is arrested for the murder."

In [0]:
generate_plot("Bella and Shelly embark on an adventure in New York City.", romance_model)

"Bella and Shelly embark on an adventure in New York City. Bella meets her ex-boyfriend, Don, and they fall in love. Don's girlfriend, Cathy, is Don's best friend and a former lover who Bella has an affair with. Bella goes back to her father's house to tell him about Don's affair and she meets Don, who has been romantically involved with Bella for a while. Don is an artist, and Bella is a pianist and dancer."

In [0]:
generate_plot("Bella marries the man of her dreams last summer on a boat.", romance_model)

'Bella marries the man of her dreams last summer on a boat. The love of her life is still waiting.\n"The film begins with the film\'s opening song, ""Sweet Honey"" by Dolly Parton, playing over a close-up of the eyes. The film\'s opening sequence is a celebration of the late Marge Simpson, the first African-American to be elected to the U.S. Congress.'

### Test Generation

In [0]:
#loading test file
test_data = pd.read_csv("/content/drive/My Drive/w266 final project/Genre_romance_test.csv")
test_data = test_data.drop(['Unnamed: 0'], axis=1)
test_data.head()

,Title,Plot,Genre
0,Cat And Mouse,Zhan Zhao is a court officer who learns of a ...,"['adventure', 'drama', 'movies', 'action', 'co..."
1,Rechipo,"Siva , a modern day Robinhood who steals money...",['romance']
2,My Chauffeur,Casey Meadows is a free-spirited young woman w...,"['indie', 'romance', 'romantic', 'comedy']"
3,Cannery Row,The story is about the skid row citizens of Mo...,"['americana', 'adaptation', 'drama', 'comedy',..."
4,Vigathakumaran,"Chandrakumar, son of a rich man in Trivandrum ...","['romance', 'drama', 'action']"


In [0]:
#sentence extraction 1: len = total len / 2
#sentence extraction 1: len = total len - 2
def extract_beginning_sent(text):
  sent_tokens = tokenize.sent_tokenize(text)
  size = max(len(sent_tokens) - 2, 1)
  beg_text = sent_tokens[:size]
  beg = " ".join(beg_text)
  return beg

def extract_beginning_word(text, threshold):
  word_tokens = tokenize.word_tokenize(text)
  size = max(int(len(word_tokens) * threshold), 1)
  beg_text = word_tokens[:size]
  beg = " ".join(beg_text)
  return beg, len(word_tokens) - size

In [0]:
#single generation
def generate_plot(prefix, num_tokens):
  output = gpt2.generate(sess,
                run_name=romance_model,
                prefix = prefix,
                length=num_tokens,
                temperature=0.7,
                top_p=0.9,
                return_as_list=True
                )
  out = output[0].rpartition('.')[0] + "."
  return out

In [0]:
len(test_data)

470

In [0]:
!mkdir generated_test_plots


In [0]:
start_point = 442
break_point = 470

In [0]:
start_time = time.time()
subset = test_data.iloc[start_point: break_point]
padding = 15

for index, row in subset.iterrows():
  if start_point == break_point:
    break
  if start_point % 10 == 0:
    print("Currently at: " + str(start_point))
  plot = row["Plot"]
  t, remaining = extract_beginning_word(plot, 0.75)
  generated_plot = generate_plot(t, remaining + padding)
  file_name = "Plot Summary: " + row["Title"] + ".txt"
  file = open("generated_test_plots/" + file_name,"w")
  file.write(generated_plot)
  file.close() 
  start_point += 1
print("My program took", time.time() - start_time,  "to run")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Currently at: 450
Currently at: 460
My program took 487.2573220729828 to run


In [0]:
file_name

'Plot Summary: The Fairy Tale Killer.txt'

In [0]:
# number of movie plots
import os
path, dirs, files_ = next(os.walk("/content/generated_test_plots"))
file_count = len(files_)
print(file_count)

470


In [0]:
!zip -r /content/test_plots.zip /content/generated_test_plots

updating: content/generated_test_plots/ (stored 0%)
updating: content/generated_test_plots/Plot Summary: Their Own Desire.txt (deflated 42%)
updating: content/generated_test_plots/Plot Summary: The Dancer Upstairs.txt (deflated 34%)
updating: content/generated_test_plots/Plot Summary: Kal Kisne Dekha.txt (deflated 47%)
updating: content/generated_test_plots/Plot Summary: Adajya.txt (deflated 34%)
updating: content/generated_test_plots/Plot Summary: Happy Together.txt (deflated 47%)
updating: content/generated_test_plots/Plot Summary: Young People Fucking.txt (deflated 49%)
updating: content/generated_test_plots/Plot Summary: Saturday Night and Sunday Morning.txt (deflated 49%)
updating: content/generated_test_plots/Plot Summary: Parenni Maya Jalaima.txt (deflated 53%)
updating: content/generated_test_plots/Plot Summary: Pagala Premi.txt (deflated 50%)
updating: content/generated_test_plots/Plot Summary: Rechipo.txt (deflated 44%)
updating: content/generated_test_plots/Plot Summary: Cor

In [0]:
#152
files.download("/content/test_plots.zip")